##Setting up Kaggle

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
! kaggle datasets download -d tongpython/cat-and-dog

In [ ]:
!unzip cat-and-dog.zip -d dataset

##CNN Model

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
training_path="/content/dataset/training_set/training_set"
test_path="/content/dataset/test_set/test_set"

In [ ]:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(training_path,image_size=(200,200),batch_size=32)

In [ ]:
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(test_path,image_size=(200,200),batch_size=32)

In [ ]:
CNNmodel = tf.keras.models.Sequential()

# Feature Extraction part
CNNmodel.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200, 3)))
CNNmodel.add(tf.keras.layers.MaxPool2D(2,2))
CNNmodel.add(tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'))
CNNmodel.add(tf.keras.layers.MaxPool2D(2,2))
CNNmodel.add(tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'))

# Neural Network - For classification
CNNmodel.add(tf.keras.layers.Flatten())
CNNmodel.add(tf.keras.layers.Dense(32, activation='relu'))
CNNmodel.add(tf.keras.layers.Dense(64, activation='relu'))
CNNmodel.add(tf.keras.layers.Dense(2))


In [ ]:
CNNmodel.summary()

In [ ]:
CNNmodel.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics = ['accuracy'])

In [ ]:
history = CNNmodel.fit(train_dataset, validation_data=test_dataset, epochs = 15 )

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.ylim([0.5,1])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model')
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.ylim([0.5,1])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

##Data Augmentation

In [ ]:
#Use ImageDataGenerator to do augmentation
dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=5,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
train_dataset_aug = dataGen.flow_from_directory(
    training_path,
    target_size = (200,200),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

In [ ]:
test_dataset_aug = dataGen.flow_from_directory(
    test_path,
    target_size = (200,200),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

In [ ]:
CNNmodel_aug = tf.keras.models.Sequential()

# Feature extraction part
CNNmodel_aug.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
CNNmodel_aug.add(tf.keras.layers.MaxPooling2D((2,2)))
CNNmodel_aug.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
CNNmodel_aug.add(tf.keras.layers.MaxPooling2D((2,2)))
CNNmodel_aug.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))

# Neural network - For classification
CNNmodel_aug.add(tf.keras.layers.Flatten())
CNNmodel_aug.add(tf.keras.layers.Dense(32, activation='relu'))
CNNmodel_aug.add(tf.keras.layers.Dense(64, activation='relu'))
CNNmodel_aug.add(tf.keras.layers.Dense(2, activation='softmax'))

#Compile
CNNmodel_aug.compile(optimizer = 'adam',
                loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = CNNmodel_aug.fit(train_dataset_aug, validation_data=test_dataset_aug, epochs = 15)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.ylim([0.5,1])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model')
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.ylim([0.5,1])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

##Output

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [ ]:
img_path = "/content/dataset/training_set/training_set/cats/cat.60.jpg"
img = image.load_img(img_path, target_size=(200, 200, 3))
plt.imshow(img)
plt.show()

In [ ]:
#img_array = image.img_to_array(img)
#img_batch = np.expand_dims(img_array, axis=0)
#img_preprocessed = preprocess_input(img_batch)

img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = CNNmodel.predict(img_array)
score = tf.nn.softmax(predictions[0])

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
#print(decode_predictions(pred, top=3)[0])
result=np.sort(score)
print(result)
result_name=np.argsort(score)
for i in result_name:
  print(class_names[i], end=" ")

print()

print(class_names[result_name[-1]]," : ",result[-1]*100)
print(class_names[result_name[-2]]," : ",result[-2]*100)

In [ ]:
print("Given picture is: ",class_names[result_name[-1]])